In [1]:
# IMPORTS
import pandas as pd
import numpy as np
import json
from fuzzywuzzy import process


/home/lebyanelm/.local/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# IMPORT DATASETS
training_data = pd.read_csv('../relative_datasets/raw/PREMIER_LEAGUE_FINAL.csv')
fixtures_data = pd.read_csv('../relative_datasets/raw/premier_league-v1.0_149.csv')

In [3]:
# EXTRACT TEAM NAMES
training_team_names = set(pd.concat([training_data.HomeTeam, training_data.AwayTeam]).unique())
fixtures_team_names = set(pd.concat([fixtures_data.homeName, fixtures_data.awayName]).unique())
training_team_names

{'Arsenal',
 'Aston Villa',
 'Barnsley',
 'Birmingham',
 'Blackburn',
 'Blackpool',
 'Bolton',
 'Bournemouth',
 'Brentford',
 'Brighton',
 'Bristol City',
 'Burnley',
 'Burton',
 'Cardiff',
 'Charlton',
 'Chelsea',
 'Coventry',
 'Crystal Palace',
 'Derby',
 'Doncaster',
 'Everton',
 'Fulham',
 'Huddersfield',
 'Hull',
 'Ipswich',
 'Leeds',
 'Leicester',
 'Liverpool',
 'Luton',
 'Man City',
 'Man United',
 'Middlesbrough',
 'Millwall',
 'Milton Keynes Dons',
 'Newcastle',
 'Norwich',
 "Nott'm Forest",
 'Peterboro',
 'Plymouth',
 'Portsmouth',
 'Preston',
 'QPR',
 'Reading',
 'Rotherham',
 'Scunthorpe',
 'Sheffield United',
 'Sheffield Weds',
 'Southampton',
 'Stoke',
 'Sunderland',
 'Swansea',
 'Tottenham',
 'Watford',
 'West Brom',
 'West Ham',
 'Wigan',
 'Wolves',
 'Wycombe',
 'Yeovil'}

In [ ]:
# CREATE A MAPPING OF TEAM NAMES TO COMBAT DIFFERENT NAME CONVENTIONS
def create_mapping(training_teams, fixtures_teams):
    mappings = {}
    for team in training_teams:
        match = process.extractOne(team, fixtures_teams)
        if match and match[1] > 80:  # Threshold for match confidence
            mappings[team] = match[0]
    return mappings

mappings = create_mapping(training_team_names, fixtures_team_names)
mappings

In [5]:
mappings = {
 'Aston Villa': 'Aston Villa',
 'Sunderland': 'Sunderland',
 'Sheffield Weds': 'Sheffield Wed',
 'Fulham': 'Fulham',
 'Southampton': 'Southampton',
 'Swansea': 'Swansea',
 'Leeds': 'Leeds United',
 'Derby': 'Derby County',
 'Man City': 'Man City',
 'Everton': 'Everton',
 'Blackburn': 'Blackburn',
 'Birmingham': 'Birmingham',
 'Huddersfield': 'Huddersfield',
 'Ipswich': 'Ipswich Town',
 'Burnley': 'Burnley',
 'Man United': 'Man United',
 'Bournemouth': 'Bournemouth',
 'Norwich': 'Norwich',
 'Watford': 'Watford',
 'Bristol City': 'Bristol City',
 'QPR': 'QPR',
 'Stoke': 'Stoke',
 'Reading': 'Reading',
 'Coventry': 'Coventry City',
 'Hull': 'Hull City',
 'West Brom': 'West Brom',
 'Blackpool': 'Blackpool',
 'Millwall': 'Millwall',
 'Leicester': 'Leicester City',
 'Cardiff': 'Cardiff',
 'Wigan': 'Wigan Athletic',
 'Peterboro': 'Peterborough',
 'Tottenham': 'Tottenham',
 'Brighton': 'Brighton Hove',
 'Arsenal': 'Arsenal',
 'Preston': 'Preston NE',
 'Plymouth': 'Plymouth Arg',
 'Crystal Palace': 'Crystal Palace',
 'Chelsea': 'Chelsea',
 'West Ham': 'West Ham',
 'Barnsley': 'Barnsley',
 'Sheffield United': 'Sheffield Utd',
 'Brentford': 'Brentford',
 'Newcastle': 'Newcastle',
 'Liverpool': 'Liverpool',
 'Rotherham': 'Rotherham Utd',
 'Luton': 'Luton Town',
 'Wolves': 'Wolverhampton',
 'Nott\'m Forest': 'Nottingham',
 'Middlesbrough': 'Middlesbrough'}


In [7]:
# FINDING UNLISTED TEAMS IN THE MAPPINGS
unlisted = []
for team in training_team_names:
    found = False
    for mapping_team in mappings:
        if mapping_team == team:
            found = True
            break
    if found == False:
        unlisted.append(team)
unlisted

['Charlton',
 'Yeovil',
 'Milton Keynes Dons',
 'Scunthorpe',
 'Burton',
 'Doncaster',
 'Bolton',
 'Portsmouth',
 'Wycombe']

In [9]:
# Convert mappings to DataFrame for merging
mapping_df = pd.DataFrame([(map[0], map[1]) for map in list(mappings.items())], columns=['training_name', 'fixture_name'])
mapping_df

# Create a combined DataFrame for home and away mappings
home_mapping_df = fixtures_data[['homeName', 'homeId']].rename(columns={'homeName': 'fixture_name', 'homeId': 'teamId'})
away_mapping_df = fixtures_data[['awayName', 'awayId']].rename(columns={'awayName': 'fixture_name', 'awayId': 'teamId'})

# Concatenate home and away mappings
combined_mapping_df = pd.concat([home_mapping_df, away_mapping_df]).drop_duplicates().reset_index(drop=True)

# Merge the mappings with combined_mapping_df to get the full mapping with IDs
full_mapping_df = mapping_df.merge(combined_mapping_df, on='fixture_name', how='left')
full_mapping_df

,training_name,fixture_name,teamId
0,Aston Villa,Aston Villa,58
1,Sunderland,Sunderland,71
2,Sheffield Weds,Sheffield Wed,345
3,Fulham,Fulham,63
4,Southampton,Southampton,340
5,Swansea,Swansea,72
6,Leeds,Leeds United,341
7,Derby,Derby County,342
8,Man City,Man City,65
9,Everton,Everton,62


In [15]:
# Now map the training data
training_dataX = training_data.copy().rename(columns={'HomeTeam': 'training_home', 'AwayTeam': 'training_away'})

# Merge to replace home team names
training_dataX = training_dataX.merge(full_mapping_df[['training_name', 'teamId']], left_on='training_home', right_on='training_name', how='left')
training_dataX = training_dataX.rename(columns={'teamId': 'HomeTeamId'}).drop(columns=['training_name'])

# Merge to replace away team names
training_dataX = training_dataX.merge(full_mapping_df[['training_name', 'teamId']], left_on='training_away', right_on='training_name', how='left')
training_dataX = training_dataX.rename(columns={'teamId': 'AwayTeamId'}).drop(columns=['training_name', 'training_home', 'training_away'])

# Display the result
print(training_dataX.dropna())

             Date HTR FTR  HomeTeamId  AwayTeamId
0      11/08/2023   A   A       328.0        65.0
1      12/08/2023   H   H        57.0       351.0
2      12/08/2023   D   D      1044.0       563.0
3      12/08/2023   H   H       397.0       389.0
4      12/08/2023   D   A        62.0        63.0
...           ...  ..  ..         ...         ...
13962    02/05/10   H   A      1138.0      1077.0
13963    02/05/10   D   A        69.0        67.0
13964    02/05/10   H   H       355.0      1081.0
13966    02/05/10   D   D       345.0       354.0
13968    02/05/10   D   D        74.0       357.0

[12750 rows x 5 columns]


In [37]:
training_dataX2 = training_dataX.dropna().drop_duplicates().reset_index(drop=True)[["Date", "HomeTeamId", "AwayTeamId", "HTR", "FTR"]]
training_dataX2.to_csv("../relative_datasets/raw/PREMIER_LEAGE_ENCODED.csv")

In [38]:
training_dataX

,Date,HTR,FTR,HomeTeamId,AwayTeamId
0,11/08/2023,A,A,328.0,65.0
1,12/08/2023,H,H,57.0,351.0
2,12/08/2023,D,D,1044.0,563.0
3,12/08/2023,H,H,397.0,389.0
4,12/08/2023,D,A,62.0,63.0
...,...,...,...,...,...
13964,02/05/10,H,H,355.0,1081.0
13965,02/05/10,A,D,NaN,351.0
13966,02/05/10,D,D,345.0,354.0
13967,02/05/10,D,D,72.0,NaN
